You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [4]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [5]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [6]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [7]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [8]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [9]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [10]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [11]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [12]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [13]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document:
Title: Unveiling the Latest Trends and Key Players in Artificial Intelligence

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in today's world
- Brief overview of the latest trends in AI

II. Key Trends in Artificial Intelligence
A. Machine Learning
- Advancements in machine learning algorithms
- Applications of machine learning in various industries
B. Natural Language Processing
- Development of chatbots and virtual assistants
- Impact of NLP on cu

I now can provide a well-written blog post in markdown format, ready for publication.

Final Answer:
# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction

Artificial Intelligence (AI) has become an integral part of our daily lives, revolutionizing industries and shaping the way we interact with technology. Defined as the simulation of human intelligence processes by machines, AI plays a crucial role in today's world by enabling machines to learn from data, adapt to new inputs, and perform tasks that typically require human intelligence. The latest trends in AI are driving innovation across various sectors, from healthcare to transportation, and are reshaping the way we live and work.

## Key Trends in Artificial Intelligence

### Machine Learning

Machine learning, a subset of AI, has seen significant advancements in recent years, with the development of sophisticated algorithms that can analyze and interpret complex data sets. From predictive analy

- Display the results of your execution as markdown in the notebook.

In [14]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction

Artificial Intelligence (AI) has become an integral part of our daily lives, revolutionizing industries and shaping the way we interact with technology. Defined as the simulation of human intelligence processes by machines, AI plays a crucial role in today's world by enabling machines to learn from data, adapt to new inputs, and perform tasks that typically require human intelligence. The latest trends in AI are driving innovation across various sectors, from healthcare to transportation, and are reshaping the way we live and work.

## Key Trends in Artificial Intelligence

### Machine Learning

Machine learning, a subset of AI, has seen significant advancements in recent years, with the development of sophisticated algorithms that can analyze and interpret complex data sets. From predictive analytics to personalized recommendations, machine learning is transforming industries such as healthcare, finance, and e-commerce by enabling more efficient decision-making processes and improving overall performance.

### Natural Language Processing

Natural Language Processing (NLP) focuses on the interaction between computers and human language, leading to the development of chatbots and virtual assistants that can understand and respond to human speech. NLP has revolutionized customer service and communication, allowing businesses to engage with customers more effectively and provide personalized experiences through automated interactions.

### Computer Vision

Advancements in computer vision technology have led to significant progress in image recognition and analysis, with applications in healthcare, security, and autonomous vehicles. From diagnosing medical conditions to enhancing surveillance systems, computer vision is enhancing our ability to process visual information and make informed decisions based on real-time data.

### Autonomous Vehicles

The growth of self-driving cars and drones represents a major milestone in the AI industry, with the potential to revolutionize transportation and logistics. While challenges such as safety and regulatory concerns remain, the opportunities presented by autonomous vehicles are driving innovation and reshaping the future of mobility.

## Key Players in the AI Industry

### Tech Giants

Leading tech companies such as Google, Amazon, Microsoft, and Apple are at the forefront of AI research and development, investing heavily in cutting-edge technologies and driving innovation across various sectors. Their contributions to the field of AI have paved the way for advancements in machine learning, NLP, and computer vision, shaping the future of intelligent systems.

### Startups

Emerging AI startups are disrupting traditional industries and driving innovation through groundbreaking technologies and solutions. From healthcare to finance, AI startups are leveraging machine learning and NLP to create new opportunities and improve existing processes, demonstrating the transformative potential of AI in the business world.

### Academic Institutions

Universities and research centers play a vital role in advancing AI technologies through collaboration between academia and industry. By fostering partnerships and conducting groundbreaking research, academic institutions are driving innovation in machine learning, computer vision, and autonomous systems, shaping the future of AI research and development.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in AI research have led to exciting developments in areas such as deep learning, reinforcement learning, and explainable AI. As the field of AI continues to evolve, discussions around ethics and regulations are gaining prominence, highlighting the importance of responsible AI deployment and governance. The impact of AI on job markets and society is also a topic of interest, with debates on the future implications of automation and AI-driven decision-making processes.

## Target Audience Analysis

Professionals in the tech industry, business owners, students, and the general audience interested in AI trends and developments can benefit from staying informed on the latest advancements in Artificial Intelligence. By understanding the key trends, key players, and noteworthy news in the AI industry, readers can gain valuable insights into the dynamic field of AI and its impact on society.

## Conclusion

In conclusion, Artificial Intelligence continues to drive innovation and shape the way we interact with technology, with key trends in machine learning, NLP, computer vision, and autonomous vehicles leading the way. The involvement of tech giants, startups, and academic institutions in AI research and development is essential for advancing AI technologies and driving innovation across various sectors. As AI continues to evolve, staying informed on the latest trends and developments in the field is crucial for individuals and organizations looking to leverage the power of AI for future growth and success.

*Stay informed on the latest trends and developments in Artificial Intelligence by subscribing to our newsletter for regular updates and insights.*

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [22]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": "Batman"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Batman.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan Document:

Title: Exploring the World of Batman: Latest Trends and Key Players

Introduction:
- Brief overview of the Batman franchise
- Mention of recent movies, comics, and merchandise

Key Points:
1. Latest Trends in Batman:
- Success of recent Batman movies such as The Dark Knight trilogy and Batman v Superman
- Growing popularity of Batman comics and graphic novels
- Introduction of new Bat-family characters like Batwoman and Batgirl

2. Key Players in the Batman Universe:
- Batman/Bruce Wayne: The iconic her

I now can give a great answer

Final Answer: 

# Exploring the World of Batman: Latest Trends and Key Players

**Introduction:**

Batman, the iconic superhero with a brooding presence, has captivated audiences worldwide for generations. From his humble beginnings in comic books to his blockbuster success on the big screen, the Dark Knight's legacy continues to thrive. With a plethora of recent movies, comics, and merchandise, Batman remains a symbol of justice and resilience that resonates with fans of all ages.

**Latest Trends in Batman:**

The recent success of Batman movies like The Dark Knight trilogy and Batman v Superman has solidified Batman's status as a cinematic powerhouse. The allure of Gotham City's dark and gritty landscape, where Batman faces off against formidable foes like the Joker and Catwoman, continues to draw in audiences. The surge in popularity of Batman comics and graphic novels further demonstrates the timeless allure of the character, with fresh narratives ca

In [23]:
Markdown(result)

# Exploring the World of Batman: Latest Trends and Key Players

**Introduction:**

Batman, the iconic superhero with a brooding presence, has captivated audiences worldwide for generations. From his humble beginnings in comic books to his blockbuster success on the big screen, the Dark Knight's legacy continues to thrive. With a plethora of recent movies, comics, and merchandise, Batman remains a symbol of justice and resilience that resonates with fans of all ages.

**Latest Trends in Batman:**

The recent success of Batman movies like The Dark Knight trilogy and Batman v Superman has solidified Batman's status as a cinematic powerhouse. The allure of Gotham City's dark and gritty landscape, where Batman faces off against formidable foes like the Joker and Catwoman, continues to draw in audiences. The surge in popularity of Batman comics and graphic novels further demonstrates the timeless allure of the character, with fresh narratives captivating readers worldwide. Moreover, the introduction of new Bat-family characters such as Batwoman and Batgirl enriches the Batman universe, offering diverse perspectives on the legendary hero's journey.

**Key Players in the Batman Universe:**

At the core of the Batman universe stands Bruce Wayne, the enigmatic billionaire who dons the mantle of Batman to combat crime and corruption. Alongside Batman are pivotal figures like Robin/Nightwing, Batman's loyal ally and eventual successor, who contributes unique skills to the fight for justice. The Joker, Batman's deranged arch-nemesis and the embodiment of chaos, presents an enduring challenge for the Caped Crusader. Catwoman, the enigmatic anti-hero and love interest of Batman, adds a layer of complexity to their intricate relationship.

**Noteworthy News in the Batman World:**

Enthusiasts of Batman can anticipate updates on upcoming movies and TV shows that expand the Batman mythos and introduce new narratives to audiences. Recent developments in Batman comic storylines promise unexpected twists that keep readers engaged, while fan theories and speculations spark lively discussions about the future of the Batman franchise. The world of Batman is ever-evolving, with fresh stories and characters enriching the superhero's enduring legacy.

**Call to Action:**

I encourage readers to delve deeper into the world of Batman by exploring a myriad of comics, movies, and merchandise that celebrate the rich history and enduring charm of the Dark Knight. Whether you're a seasoned fan or a newcomer to the Batman universe, there's always something captivating to uncover and appreciate. Share your beloved Batman moments or characters in the comments section and engage in conversations about the world's greatest detective.

In conclusion, Batman stands as a cultural icon that continues to captivate fans worldwide with his compelling narratives and enduring legacy. With a diverse cast of characters, gripping storylines, and thrilling adventures, the Batman franchise serves as a beacon of hope and valor in a world fraught with darkness. Embrace the legacy of the Dark Knight and become part of a community that celebrates the indomitable spirit of one man's quest for justice.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).